## Waiting for processes

* Wait for any child to finish — `int wait(int *status)`
    * Returns PID of the child process that finished
    * `status` parameter is optional
        * if passed a valid integer pointer, `wait` will store the return value of the child process's `main` function at the referenced memory location
        * if return value of child process's main function is not needed, then pass `NULL`
* Wait for a specific process to finish — `int waitpid(pid_t pid, int *status, int options)`
    * Returns PID of the process that finished
    * `pid` is PID if process to wait for — need not be a child process
    * `status` is the same as `wait`
    * `options` is typically `0`, except in special circumstances
* Wait functions do not return until child or specific process, respectively, finishes

* Q4: _What are all possible outputs of this program?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
    } else {
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

Child
Parent


<p style="height:12em;"></p>

* Q5: _What are all possible outputs of this program (assuming the new process has PID 13346)?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    printf("A %d\n", pid);
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("B\n");
    } else {
        wait(NULL);
        printf("C\n");
    }
}

```
A 0
A 13346
B
C
```
OR
```
A 13346
A 0
B
C
```
OR
```
A 0
B
A 13346
C
```

## Extra practice

* Q5: _What are all possible outputs produced by this program?_
    ```C
    #include <stdio.h>
    #include <stdlib.h>
    #include <unistd.h>
    #include <sys/wait.h>
    int main() {
        int pid = fork();
        if (pid == 0) {
            printf("Child\n");
            exit(22);
        } else {
            int status = 0;
            wait(&status);
            printf("Status %d\n", WEXITSTATUS(status));
            exit(44);
        }
    }
    ```
    ```
    Child
    Status 22
    ```

* Q8: _How would you modify the program below such that `Child` always prints before `Parent`?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
        return 0;
    } else {
        printf("Parent\n");
    }
    printf("End\n");
}

In [17]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
        return 0;
    } else {
        wait(NULL);
        printf("Parent\n");
    }
    printf("End\n");
}

Begin
Child
Parent
End


## Warm-up: Limited Direct Execution (LDE)

* Q1: _One alternative to Limited Direct Execution (LDE) is **indirect execution**—i.e., for each assembly instruction an application wants to execute, the application must ask the OS to execute the instruction on the processor on the application’s behalf. Why is indirect execution a **poor design choice**?_

    Significant overhead

<div style="height:7em;"></div>

* Q2: _Another alternative to LDE is **direct execution**–i.e., an application can execute its assembly instructions directly on the processor and access any location in main memory. Why is direct execution a **poor design choice**?_

    An application can interfere with another application's memory; an application can use the processor indefinitely

<div style="height:7em;"></div>

* Q3: _What **processor feature(s)** make(s) LDE possible?_

    Two modes: kernel mode (processor is in this mode when the OS is running) and user mode (processor is in this mode when the application is running); `svc` instruction to make a system call; time interrupts 

<div style="height:7em;"></div>

* Q4: _What work (if any) must be done by an application, the operating system (OS), and the processor when an application wants to perform a privileged operation—e.g., start another application?_

    Application puts the system call number in a register and executes the `svc` instruction; Processor switches to kernel mode, saves the values in the registers on the OS's stack, and branches to the appropriate system call handling code based on the system call number; OS executes the system call; Processor restores the application's register values from the OS's stack, switches to user mode, and resumes execution of the application by executing the next instruction after the `svc` instruction

<div style="height:15em;"></div>

🛑 **STOP here** after completing the above questions; if you have extra time please **take a few deep breaths** to the reduce stress.

<div style="page-break-after:always;"></div>

## Creating processes

* `int fork()`
    * Creates an exact copy of the running process, except for the return value from `fork` — return `0` to child (i.e., new) process; return child’s process ID to parent process (i.e., process that called fork)
    * Both child and parent resume execution from place where `fork` was called

* Q6: _What does the following code output?_

In [4]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    int x = 13;
    printf("Before fork %d\n", x);
    fork();
    printf("After fork %d\n", x);
    return 0;
}

Before fork 13
After fork 13
After fork 13


<div style="page-break-after:always;"></div>

* Q7: _What does the following code output (assuming the new process has PID 1819)?_

In [3]:
#include <stdio.h>
#include <unistd.h>    
int main(int argc, char **argv) {
    printf("Before fork\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child gets %d\n", pid);
    } else {
        printf("Parent gets %d\n", pid);
    }
    return 0;
}

Before fork
Parent gets 2976
Child gets 0


_Postponed to next class_

<p style="height:10em;"></p>

## Extra practice

* Q8: _What does the following code output?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    int value = 100;
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        value -= 50;
    } else {
        value += 50;
    }
    printf("My value is %d\n", value);
    return 0;
}

```
My value is 50
My value is 150
```
OR
```
My value is 150
My value is 50
```

<p style="height:12em;"></p>

* Q9: _What does the following code output?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
        return 0;
    } else {
        printf("Parent\n");
    }
    printf("End\n");
}

```
Begin
Child
Parent
End 
```
OR
```
Begin
Parent
Child
End 
```
OR
```
Begin
Parent
End 
Child
```